In [1]:
import pandas as pd
import numpy as np
import fastf1
import datetime
fastf1.Cache.enable_cache('cache') 

In [2]:
def find_fastest_laps(laps, cols, index_cols, lap_num):
    
    laps = laps.sort_values('LapTime')
    fastest = laps.groupby('DriverNumber').nth(0).reset_index()

    fastest_time = {}
    added_cols = []
    
    
    for col in cols:
        fastest_time[col] = laps[col].min()
        
        added_cols.append(col + 'Percent')
       # added_cols.append(col[:-4] + 'Position')
        # fastest[col[:-4] + 'Percent'] = (fastest[col] - fastest_time) / fastest_time
        # total_cols.append(col[:-4] + 'Percent')
    
    # dummies = pd.get_dummies(fastest['Compound'])
    # #dummies.columns = dummies.columns + '_1'
    # fastest = pd.concat([fastest, dummies], axis = 1)
    
    fastest = fastest[index_cols]
    

    # top_cols = ['Sector1Time', 'Sector2Time', 'Sector3Time']
    # fastest = fastest.merge(laps.groupby('DriverNumber').min()[top_cols].reset_index(), on=['DriverNumber'], how = 'outer')
    for i in range(0,lap_num):
        laps_i = laps.groupby('DriverNumber').nth(i).reset_index()

        for col in cols:
            laps_i[col + 'Percent'] = (laps_i[col] - fastest_time[col]) / fastest_time[col]
            #laps_i[col[:-4] + 'Position'] = laps_i.sort_values(col).index
        # laps_i['LapPercent'] = (laps_i['LapTime'] - fastest_lap) / fastest_lap
        fastest = fastest.merge(laps_i[index_cols + cols + added_cols], on=index_cols, how = 'outer', suffixes=('', '_' + str(i)))
    return fastest

In [3]:
def load_practice(event, type, year):
    
    practice1 = fastf1.get_session(year, event, 'Practice 1')
    practice1.load(laps=True,messages=True)
    try:
        laps1 = practice1.laps
        cols = ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time','TyreLife','SpeedI1','SpeedI2','SpeedFL']
        index_cols = ['DriverNumber', 'Team', 'Driver']
        practice1 = find_fastest_laps(laps1, cols, index_cols, 2)
        practice2 = practice1
        if type == 'conventional':
            practice2 = fastf1.get_session(year, event, 'Practice 2')
            practice2.load(laps=True,messages=True)
            laps2 = practice2.laps
            practice2 = find_fastest_laps(laps2, cols, index_cols, 2)

        practice_data = practice1.merge(practice2, on= index_cols, how = 'outer', suffixes=('_1', '_2'))
        return practice_data
    except:
         return pd.DataFrame()
   
def load_qualifying(event, year):
        qualifying = fastf1.get_session(year, event, 'Q')
        qualifying.load()
        try:
            results = qualifying.results

            results['FastestLap'] = results[['Q1', 'Q2', 'Q3']].apply(lambda x: min(x), axis = 1)

            fastest_lap_q = results['FastestLap'].min()
            results['LapTimePercent'] = (results['FastestLap'] - fastest_lap_q)/ fastest_lap_q
            results['Team'] = results['TeamName']
            return results[['DriverNumber', 'Position', 'LapTimePercent', 'Team']]
        except:
             return pd.DataFrame()


In [4]:
years = [2018, 2019,2020, 2021, 2022, 2023]
# years = [2022, 2023]

intitial_data = pd.DataFrame()
#prev_qualifying_data = pd.DataFrame()
for year in years:
    schedule = fastf1.get_event_schedule(year)
    for index, row in schedule.iterrows():
        if row['EventFormat'] in ['conventional']:
            practice_data = load_practice(row['EventName'], row['EventFormat'], year)
            qualifying_data = load_qualifying(row['EventName'], year)
            if 'DriverNumber' in practice_data.columns and 'DriverNumber' in qualifying_data.columns:
                #if 'DriverNumber' in prev_qualifying_data.columns:
                    # qualifying_data_full = qualifying_data.merge(prev_qualifying_data, on = 'DriverNumber', suffixes=('_curr', '_prev'))
                full_data = practice_data.merge(qualifying_data, on=['DriverNumber', 'Team'])
                for col in ['RoundNumber', 'Country', 'Location', 'OfficialEventName', 'EventDate','EventName', 'EventFormat']:
                    full_data[col] = row[col]
                intitial_data = pd.concat([intitial_data, full_data], axis = 0, ignore_index=True)
                # prev_qualifying_data = qualifying_data

core           INFO 	Loading data for Australian Grand Prix - Practice 1 [v2.3.1]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df,

In [5]:
data = pd.DataFrame(intitial_data)
prev_range = 5

In [6]:
data.columns

Index(['DriverNumber', 'Team', 'Driver', 'LapTime_1', 'Sector1Time_1',
       'Sector2Time_1', 'Sector3Time_1', 'TyreLife_1', 'SpeedI1_1',
       'SpeedI2_1', 'SpeedFL_1', 'LapTimePercent_1', 'Sector1TimePercent_1',
       'Sector2TimePercent_1', 'Sector3TimePercent_1', 'TyreLifePercent_1',
       'SpeedI1Percent_1', 'SpeedI2Percent_1', 'SpeedFLPercent_1',
       'LapTime_1_1', 'Sector1Time_1_1', 'Sector2Time_1_1', 'Sector3Time_1_1',
       'TyreLife_1_1', 'SpeedI1_1_1', 'SpeedI2_1_1', 'SpeedFL_1_1',
       'LapTimePercent_1_1', 'Sector1TimePercent_1_1',
       'Sector2TimePercent_1_1', 'Sector3TimePercent_1_1',
       'TyreLifePercent_1_1', 'SpeedI1Percent_1_1', 'SpeedI2Percent_1_1',
       'SpeedFLPercent_1_1', 'LapTime_2', 'Sector1Time_2', 'Sector2Time_2',
       'Sector3Time_2', 'TyreLife_2', 'SpeedI1_2', 'SpeedI2_2', 'SpeedFL_2',
       'LapTimePercent_2', 'Sector1TimePercent_2', 'Sector2TimePercent_2',
       'Sector3TimePercent_2', 'TyreLifePercent_2', 'SpeedI1Percent_2',
      

In [7]:
team_cols = []
individual_cols = []


for i in range(prev_range):
    data['PrevLapTimePercent' + str(i+1)] = data.groupby('DriverNumber')['LapTimePercent'].shift(i+1)
    individual_cols.append('PrevLapTimePercent' + str(i+1))
    # data['PrevPosition' + str(i+1)] = data.groupby('DriverNumber')['Position'].shift(i+1)
    prev = data.groupby(['Team', 'OfficialEventName', 'EventDate'])[['Position', 'LapTimePercent']].mean().reset_index().sort_values('EventDate').groupby('Team').shift(i+1)
    prev.index = data.groupby(['Team', 'OfficialEventName', 'EventDate'])[['Position', 'LapTimePercent']].mean().reset_index(2).sort_values('EventDate').index
    prev = prev[['Position', 'LapTimePercent']].reset_index()
    data = data.merge(prev, on = ['OfficialEventName', 'Team'], suffixes=('', 'PrevTeam' + str(i+1)))
    team_cols.append('LapTimePercentPrevTeam' + str(i+1))
    #team_cols.append('PositionPrevTeam' + str(i+1))

In [8]:
def cap_outliers(row):
    median = np.median(row)
    return np.clip(row, median-0.02, median+0.02)

In [9]:
data.columns

Index(['DriverNumber', 'Team', 'Driver', 'LapTime_1', 'Sector1Time_1',
       'Sector2Time_1', 'Sector3Time_1', 'TyreLife_1', 'SpeedI1_1',
       'SpeedI2_1', 'SpeedFL_1', 'LapTimePercent_1', 'Sector1TimePercent_1',
       'Sector2TimePercent_1', 'Sector3TimePercent_1', 'TyreLifePercent_1',
       'SpeedI1Percent_1', 'SpeedI2Percent_1', 'SpeedFLPercent_1',
       'LapTime_1_1', 'Sector1Time_1_1', 'Sector2Time_1_1', 'Sector3Time_1_1',
       'TyreLife_1_1', 'SpeedI1_1_1', 'SpeedI2_1_1', 'SpeedFL_1_1',
       'LapTimePercent_1_1', 'Sector1TimePercent_1_1',
       'Sector2TimePercent_1_1', 'Sector3TimePercent_1_1',
       'TyreLifePercent_1_1', 'SpeedI1Percent_1_1', 'SpeedI2Percent_1_1',
       'SpeedFLPercent_1_1', 'LapTime_2', 'Sector1Time_2', 'Sector2Time_2',
       'Sector3Time_2', 'TyreLife_2', 'SpeedI1_2', 'SpeedI2_2', 'SpeedFL_2',
       'LapTimePercent_2', 'Sector1TimePercent_2', 'Sector2TimePercent_2',
       'Sector3TimePercent_2', 'TyreLifePercent_2', 'SpeedI1Percent_2',
      

In [10]:
data[team_cols] = data[team_cols].apply(cap_outliers, axis=1, result_type='expand')
data[individual_cols] = data[individual_cols].apply(cap_outliers, axis=1, result_type='expand')


In [11]:
data['LapTimePrevPercentTeamAverage'] = data[team_cols].apply(sum, axis=1)

In [12]:
data.columns

Index(['DriverNumber', 'Team', 'Driver', 'LapTime_1', 'Sector1Time_1',
       'Sector2Time_1', 'Sector3Time_1', 'TyreLife_1', 'SpeedI1_1',
       'SpeedI2_1', 'SpeedFL_1', 'LapTimePercent_1', 'Sector1TimePercent_1',
       'Sector2TimePercent_1', 'Sector3TimePercent_1', 'TyreLifePercent_1',
       'SpeedI1Percent_1', 'SpeedI2Percent_1', 'SpeedFLPercent_1',
       'LapTime_1_1', 'Sector1Time_1_1', 'Sector2Time_1_1', 'Sector3Time_1_1',
       'TyreLife_1_1', 'SpeedI1_1_1', 'SpeedI2_1_1', 'SpeedFL_1_1',
       'LapTimePercent_1_1', 'Sector1TimePercent_1_1',
       'Sector2TimePercent_1_1', 'Sector3TimePercent_1_1',
       'TyreLifePercent_1_1', 'SpeedI1Percent_1_1', 'SpeedI2Percent_1_1',
       'SpeedFLPercent_1_1', 'LapTime_2', 'Sector1Time_2', 'Sector2Time_2',
       'Sector3Time_2', 'TyreLife_2', 'SpeedI1_2', 'SpeedI2_2', 'SpeedFL_2',
       'LapTimePercent_2', 'Sector1TimePercent_2', 'Sector2TimePercent_2',
       'Sector3TimePercent_2', 'TyreLifePercent_2', 'SpeedI1Percent_2',
      

In [13]:
data.to_pickle('train_data.pkl')

In [17]:
sched_2023 = fastf1.get_event_schedule(2023)
round_number = 10

In [18]:
pred_event = sched_2023[sched_2023['RoundNumber'] == round_number]

In [19]:
pred_data = load_practice(pred_event['EventName'].iloc[0], pred_event['EventFormat'].iloc[0], 2023)
for col in ['RoundNumber', 'Country', 'Location', 'OfficialEventName', 'EventDate','EventName', 'EventFormat']:
    pred_data[col] = pred_event[col].iloc[0]
i = 0
while i < prev_range:
    prev_event = sched_2023[sched_2023['RoundNumber'] == round_number - i - 1]
    prev_data = load_qualifying(prev_event['EventName'].iloc[0], 2023)
    if prev_data.shape[0] != 0:
        prev_data['PrevLapTimePercent' + str(i+1)] = prev_data['LapTimePercent']
        # prev_data['PrevPosition' + str(i+1)] = prev_data['Position']
        prev_data = prev_data.merge(prev_data.groupby('Team')[['LapTimePercent', 'Position']].mean().reset_index(), on=['Team'], suffixes=('', 'PrevTeam' + str(i+1)))

        pred_data = pred_data.merge(prev_data[['DriverNumber', 'PrevLapTimePercent' + str(i+1), 'LapTimePercentPrevTeam' + str(i+1)]], on = 'DriverNumber')
        i += 1

core           INFO 	Loading data for British Grand Prix - Practice 1 [v2.3.1]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1492: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for key, value in row.iteritems():
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packages/fastf1/core.py:1415: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, result], sort=False)
/Users/jamisonmeindl/opt/anaconda3/lib/python3.8/site-packag

In [20]:
pred_data

,DriverNumber,Team,Driver,LapTime_1,Sector1Time_1,Sector2Time_1,Sector3Time_1,TyreLife_1,SpeedI1_1,SpeedI2_1,...,PrevLapTimePercent1,LapTimePercentPrevTeam1,PrevLapTimePercent2,LapTimePercentPrevTeam2,PrevLapTimePercent3,LapTimePercentPrevTeam3,PrevLapTimePercent4,LapTimePercentPrevTeam4,PrevLapTimePercent5,LapTimePercentPrevTeam5
0,1,Red Bull Racing,VER,0 days 00:01:28.600000,0 days 00:00:28.147000,0 days 00:00:35.994000,0 days 00:00:24.459000,5.0,322.0,266.0,...,0.000000,0.006103,0.004662,0.016520,0.000000,0.007347,0.000000,0.017410,0.000000,0.000156
1,10,Alpine,GAS,0 days 00:01:29.828000,0 days 00:00:28.380000,0 days 00:00:36.317000,0 days 00:00:25.131000,2.0,311.0,263.0,...,0.012098,0.014295,0.052855,0.037104,0.007527,0.008807,0.007959,0.005297,0.009192,0.008224
2,11,Red Bull Racing,PER,0 days 00:01:29.048000,0 days 00:00:28.150000,0 days 00:00:36.257000,0 days 00:00:24.641000,5.0,322.0,265.0,...,0.012207,0.006103,0.028377,0.016520,0.014694,0.007347,0.034821,0.017410,0.000311,0.000156
3,14,Aston Martin,ALO,0 days 00:01:29.268000,0 days 00:00:28.199000,0 days 00:00:36.282000,0 days 00:00:24.787000,2.0,317.0,264.0,...,0.008076,0.007936,0.013350,0.024198,0.011429,0.010710,0.001177,0.009402,0.003260,0.011202
4,16,Ferrari,LEC,0 days 00:01:29.280000,0 days 00:00:28.255000,0 days 00:00:36.240000,0 days 00:00:24.785000,5.0,315.0,263.0,...,0.000745,0.001848,0.024008,0.019187,0.025003,0.015698,0.001485,0.002599,0.001728,0.002788
5,18,Aston Martin,STR,0 days 00:01:29.471000,0 days 00:00:28.183000,0 days 00:00:36.240000,0 days 00:00:25.048000,2.0,319.0,264.0,...,0.007796,0.007936,0.035046,0.024198,0.009990,0.010710,0.017628,0.009402,0.019144,0.011202
6,2,Williams,SAR,0 days 00:01:30.124000,0 days 00:00:28.337000,0 days 00:00:36.738000,0 days 00:00:25.049000,6.0,320.0,262.0,...,0.024180,0.019824,0.058584,0.029292,0.033581,0.029181,0.024494,0.020388,0.020308,0.015804
7,20,Haas F1 Team,MAG,0 days 00:01:30.385000,0 days 00:00:28.525000,0 days 00:00:36.837000,0 days 00:00:25.023000,2.0,317.0,260.0,...,0.024538,0.017697,0.037510,0.028790,0.024491,0.018866,0.026694,0.026757,0.009895,0.011219
8,21,AlphaTauri,DEV,0 days 00:01:29.691000,0 days 00:00:28.239000,0 days 00:00:36.518000,0 days 00:00:24.934000,2.0,321.0,261.0,...,0.024584,0.023109,0.056043,0.053560,0.018112,0.020056,0.014895,0.012471,0.017405,0.018004
9,22,AlphaTauri,TSU,0 days 00:01:30.092000,0 days 00:00:28.432000,0 days 00:00:36.609000,0 days 00:00:25.051000,2.0,318.0,265.0,...,0.021633,0.023109,0.051077,0.053560,0.022000,0.020056,0.010047,0.012471,0.018603,0.018004


In [21]:
pred_data['LapPercentPrevTeamAverage'] = pred_data[team_cols].apply(sum, axis=1)

In [22]:
pred_data.to_pickle('pred_data.pkl')